In [ ]:
from scaling_function import *
from scipy.integrate import quad

In [ ]:
# primal order
d = 3
# dual order
d_t = 5 

# Scaling Function

In [ ]:
# initialization
sf = PrimalScalingFunction(d)
# visualization (see [3, Figure 2.11-2.14])
sf.plot()

In [ ]:
# approximate L1-norm from numerical quadrature
l1_norm = quad(sf, sf.l1, sf.l2)[0]
print("L1 norm =", l1_norm)
# verification of [3, (2.38)]
assert np.isclose(l1_norm, 1.)

# approximate L2-norm from numerical quadrature
l2_norm = np.sqrt(quad(lambda x: sf(x)**2, sf.l1, sf.l2)[0])
print("L2 norm =", l2_norm)

In [ ]:
# normalization by L2-norm (see [3, Figure 5.5-5.11])
sf.plot(scale=1./l2_norm)

In [ ]:
# verification of partition of unity [3, Proposition 2.10]
nx = 200
x = np.linspace(-d, d, nx)
s = np.zeros(nx)
for k in range(sf.l1, sf.l2 + 1):
    s += sf(x + k)
plt.plot(x, s)
plt.show()
assert np.isclose(s[nx//2], 1.)

In [ ]:
# visualization of derivative
sf.plot(nu=1)

# approximate H1-norm from numerical quadrature
sf_deriv = sf.derivative()
h1_norm = np.sqrt(quad(lambda x: sf_deriv(x)**2, sf.l1, sf.l2)[0] + l2_norm**2)
print("H1 norm =", h1_norm)

In [ ]:
# refinement coefficients (see [3, Table 2.6-2.7])
a = sf.refinement_coeffs()

# verification of [3, Proposition 2.7]
sum_a = a.sum()
print("refinement coefficients =", a)
print("sum =", sum_a)
assert np.isclose(sum_a, 2.)

In [ ]:
# entries of Gramian matrix
g = sf.gramian()

# approximate L2-norm from Gramian matrix
l2_norm_g = np.sqrt(g[d-1])
print("L2 norm from gramian =", l2_norm_g)
print("L2 norm from quadrature =", l2_norm)
assert np.isclose(l2_norm_g, l2_norm)

In [ ]:
# validation of Gramian matrix for d = 3
# reference value from [2, Section 5.1]
g_test = PrimalScalingFunction(3).gramian()
g_ref = np.array([1/120, 13/60, 11/20, 13/60, 1/120])
assert np.allclose(g_test, g_ref)

In [ ]:
# entries of stiffness matrix
h = sf.inner_product(nu=1)

# approximate H1-norm from Gramian and stiffness matrix
h1_norm_h = np.sqrt(h[d-1] + g[d-1])
print("H1 norm from inner product =", h1_norm_h)
print("H1 norm from quadrature =", h1_norm)
assert np.isclose(h1_norm_h, h1_norm)

In [ ]:
# validation of first order stiffness matrix for d = 3
# reference value from [1, Beispiel 3.26]
h_test = PrimalScalingFunction(3).inner_product(nu=1)
h_ref = np.array([-32/3, -64/3, 64, -64/3, -32/3]) / 2**6
assert np.allclose(h_test, h_ref)

# Dual Scaling Function
Due to low regularity of dual scaling functions, results given by numerical quadrature are unreliable .

In [ ]:
# initialization
sf_t = DualScalingFunction(d, d_t)
# visualization (see [3, Figure 2.11-2.14])
sf_t.plot()

In [ ]:
# approximate L1-norm from numerical quadrature
l1_norm = quad(sf_t, sf_t.l1, sf_t.l2)[0]
print("L1 norm =", l1_norm)
# verification of [3, (2.38)]
# assert np.isclose(l1_norm, 1.)

# approximate L2-norm from numerical quadrature
l2_norm = np.sqrt(quad(lambda x: sf_t(x)**2, sf_t.l1, sf_t.l2)[0])
print("L2 norm =", l2_norm)

In [ ]:
# normalization by L2-norm (see [3, Figure 5.5-5.11])
sf_t.plot(scale=1./l2_norm)

In [ ]:
# verification of partition of unity [3, Proposition 2.10]
nx = 200
x = np.linspace(-d-2*(d_t-1), d+2*(d_t-1), nx)
s = np.zeros(nx)
for k in range(sf_t.l1, sf_t.l2 + 1):
    s += sf_t(x + k)
plt.plot(x, s)
plt.show()
# assert np.isclose(s[nx//2], 1.)

In [ ]:
# refinement coefficients (see [3, Table 2.6-2.7])
a = sf_t.refinement_coeffs()

# verification of [3, Proposition 2.7]
sum_a = a.sum()
print("refinement coefficients =", a)
print("sum =", sum_a)
assert np.isclose(sum_a, 2.)

In [ ]:
# verification of biorthogonality
res = []
for k in range(sf.l1-sf_t.l2+1, sf.l2-sf_t.l1):
    res.append(quad(lambda x: sf(x)*sf_t(x-k), sf.l1, sf.l2)[0])
res = np.array(res)
print("biorthogonality:")
print(res)

exact = np.zeros_like(res)
exact[res.size//2] = 1.
print("expected values:")
print(exact)
# assert np.allclose(res, exact, atol=1e-3)

In [ ]:
# entries of Gramian matrix
g = sf_t.gramian()

# approximate L2-norm from Gramian matrix
l2_norm_g = np.sqrt(g[d+2*d_t-3])
print("L2 norm from gramian =", l2_norm_g)
print("L2 norm from quadrature =", l2_norm)
# assert np.isclose(l2_norm_g, l2_norm)

In [ ]:
# validation of Gramian matrix for d = 2, d_t = 2
# reference value from [1, Beispiel 4.25]
g_test = DualScalingFunction(2, 2).gramian()
g_ref = np.array([1/288, 1/8, -67/96, 77/36, -67/96, 1/8, 1/288])
assert np.allclose(g_test, g_ref)

# Mother Wavelet

In [ ]:
# initialization
mw = MotherWavelet(d, d_t)
# visualization
mw.plot()

In [ ]:
# approximate L1-norm from numerical quadrature
l1_norm = quad(mw, mw.l1, mw.l2)[0]
print("L1 norm =", l1_norm)
assert np.isclose(l1_norm, 0.)

# approximate L2-norm from numerical quadrature
l2_norm = np.sqrt(quad(lambda x: mw(x)**2, mw.l1, mw.l2)[0])
print("L2 norm =", l2_norm)

In [ ]:
# normalization by L2-norm (see [3, Figure 5.5-5.11])
mw.plot(scale=1./l2_norm)

In [ ]:
# visualization of derivative
mw.plot(nu=1)

# approximate H1-norm from numerical quadrature
mw_deriv = mw.derivative()
h1_norm = np.sqrt(quad(lambda x: mw_deriv(x)**2, mw.l1, mw.l2)[0] + l2_norm**2)
print("H1 norm =", h1_norm)

In [ ]:
# entries of Gramian matrix
g = mw.gramian()

# approximate L2-norm from Gramian matrix
l2_norm_g = np.sqrt(g[d+d_t-2])
print("L2 norm from gramian =", l2_norm_g)
print("L2 norm from quadrature =", l2_norm)
assert np.isclose(l2_norm_g, l2_norm)

In [ ]:
# entries of stiffness matrix
h = mw.inner_product(nu=1)

# approximate H1-norm from Gramian and stiffness matrix
h1_norm_h = np.sqrt(h[d+d_t-2] + g[d+d_t-2])
print("H1 norm from inner product =", h1_norm_h)
print("H1 norm from quadrature =", h1_norm)
assert np.isclose(h1_norm_h, h1_norm)

# References
[1] Primbs, Miriam. 2006. “Stabile biortogonale Spline-Waveletbasen auf dem Intervall.”

[2] Primbs, Miriam. 2008. “On the Computation of Gramian Matrices for Refinable Bases on the Interval.” International Journal of Wavelets, Multiresolution and Information Processing 06 (03): 459–79. https://doi.org/10.1142/S0219691308002422.

[3] Urban, Karsten. 2009. Wavelet Methods for Elliptic Partial Differential Equations. Numerical Mathematics and Scientific Computation. Oxford ; New York: Oxford University Press.